# Q2

In [302]:
from scipy.io import loadmat
import os
import numpy as np
import numpy.linalg as npla

In [300]:
# Import wine data
wine = loadmat('wine.mat')
traindata = wine['data']
trainlabels = wine['labels'].astype('float')
testdata = wine['testdata']
testlabels = wine['testlabels'].astype('float')

# Store variable names
variables = ['intercept',
             'fixed acidity',
             'volatile acidity',
             'citric acid',
             'residual sugar',
             'chlorides',
             'free sulfur dioxide',
             'total sulfur dioxide',
             'density',
             'pH',
             'sulphates',
             'alcohol']

# Store number of variables
numVariables = traindata.shape[1]

## Ordinary Least Squares

In [303]:
OLS_betas = npla.inv(traindata.T @ traindata) @ traindata.T @ trainlabels
OLS_yhat = testdata @ OLS_betas
OLS_MSE = (OLS_yhat.T @ OLS_yhat - 2*(OLS_yhat.T @ testlabels) + testlabels.T @ testlabels)/len(OLS_yhat)

In [336]:
print("Test squared loss risk is {:.5f}.".format(OLS_MSE.item()))

Test squared loss risk is 0.53978.


## Sparse Linear Predictor

In [305]:
# Run sparse linear predictor
SLP_minMSE = 1000000000
SLP_minMSEIndex = [0,1,2,3]
SLP_minMSEBetas = []

for i in range(1,numVariables):
    for j in range(i+1,numVariables):
        for k in range(j+1,numVariables):

            # Define training data and betas
            x = traindata[:,[0,i,j,k]]
            y = trainlabels
            SLP_betas = npla.inv(x.T @ x) @ x.T @ y

            # Define test data and predictions
            SLP_yhat = x @ SLP_betas

            # Calculate MSE
            tmpMSE = (SLP_yhat.T @ SLP_yhat - 2*(SLP_yhat.T @ y) + y.T @ y)/len(ypred)

            # Store argminimizer of MSE
            if tmpMSE < SLP_minMSE:
                SLP_minMSE=tmpMSE
                SLP_minMSEIndex=[0,i,j,k]
                SLP_minMSEBetas=SLP_betas
                
# List of variables with lowest MSE
SLP_minMSEVariables=[variables[v] for v in SLP_minMSEIndex]

In [334]:
print("The sparse linear predictor results in {} with a MSE of {:.5f}.".format(SLP_minMSEVariables[1:],SLP_minMSE.item()))

The sparse linear predictor results in ['volatile acidity', 'sulphates', 'alcohol'] with a MSE of 0.55877.


In [306]:
SLP_yhat = testdata[:,SLP_minMSEIndex] @ SLP_betas
SLP_MSE = (SLP_yhat.T @ SLP_yhat - 2*(SLP_yhat.T @ testlabels) + testlabels.T @ testlabels)/len(SLP_yhat)

In [337]:
print("Test squared loss risk is {:.4f}.".format(SLP_MSE.item()))

Test squared loss risk is 0.5942.


## Highest Correlated Variables

In [332]:
# Closest Covariance
closestCov = dict()
covMatrix = np.cov(testdata.T)

for i in range(1,numVariables):
    # Finds the indexes with the top 3 absolute covariance
    covargmax = np.argsort([abs(i) for i in covMatrix[i]])[-3:][::-1]
    
    # Add each variable to a dictionary - Add a dictionary of the top two covarying variables and their covariance
    closestCov[variables[i]]={variables[covargmax[1]]:covMatrix[i][covargmax[1]],
                              variables[covargmax[2]]:covMatrix[i][covargmax[2]]}


    

## Solutions

#### a)

In [327]:
print("The risks for the ordinary least squares estimator is {:.4f}.".format(OLS_MSE.item()))
print("The risks for the sparse linear predictor is {:.4f}.".format(SLP_MSE.item()))

The risks for the ordinary least squares estimator is 0.5398.
The risks for the sparse linear predictor is 0.5942.


#### b)

In [340]:
print("{:<7} {:<20} {:<10}".format('Index','Variable','Coefficient'))
print("-"*40)

for i in range(4):
    print(" {:<7} {:<20} {:.5f}".format(SLP_minMSEIndex[i], SLP_minMSEVariables[i],SLP_betas[i].item()))


Index   Variable             Coefficient
----------------------------------------
 0       intercept            5.82610
 2       volatile acidity     -0.03025
 10      sulphates            0.05528
 11      alcohol              0.38749


#### c)

In [333]:
print("{:<25} {:<8} {:<25} {:<10}".format('Variable','Rank','Most Correlated With','Covariance'))
print("-"*71)

for i in closestCov.keys():
    rank=0
    for j in closestCov[i].keys():
        rank+=1
        print(" {:<25} {:<8} {:<25} {:.5f}".format(i, rank, j, closestCov[i][j]))
    print("-"*71)
    
print("\n  Note: the affine expansion variable is not included, because it has a\n\tcovariance of 0 with all variables.")

Variable                  Rank     Most Correlated With      Covariance
-----------------------------------------------------------------------
 fixed acidity             1        density                   0.46083
 fixed acidity             2        citric acid               0.35730
-----------------------------------------------------------------------
 volatile acidity          1        total sulfur dioxide      -0.42338
 volatile acidity          2        citric acid               -0.40332
-----------------------------------------------------------------------
 citric acid               1        volatile acidity          -0.40332
 citric acid               2        pH                        -0.36184
-----------------------------------------------------------------------
 residual sugar            1        density                   0.60091
 residual sugar            2        total sulfur dioxide      0.49291
-----------------------------------------------------------------------
 chl